## Imports

In [1]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import time
import copy
import tqdm.notebook as tqdm

# Pytorch package
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision.transforms as T
from torchvision.models.feature_extraction import get_graph_node_names, create_feature_extractor

#Image packages
from PIL import Image

In [2]:
# Check device availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("You are using device: %s" % device)

You are using device: cuda


## Load Data

In [3]:
mydir = os.getcwd()
train_dir = os.path.join(mydir, 'chest_xray', 'train')
test_dir = os.path.join(mydir, 'chest_xray', 'test')

In [4]:
mean_nums = [0.485, 0.456, 0.406]
std_nums = [0.229, 0.224, 0.225]

cnn_transforms =  T.Compose([
                        T.Resize(256),
                        T.CenterCrop(224),
                        T.ToTensor(),
                        T.Normalize(mean=mean_nums, std=std_nums),
                    ])

# data_transforms = {"train": transforms.Compose([
#                                             transforms.Resize(256),
#                                             transforms.CenterCrop(224),
#                                             transforms.ToTensor(),
#                                             transforms.Normalize(mean=mean_nums, std=std_nums),
#                                         ]),
#                     "test": transforms.Compose([
#                                 transforms.Resize(256),
#                                 transforms.CenterCrop(224),
#                                 transforms.ToTensor(),
#                                 transforms.Normalize(mean=mean_nums, std=std_nums),
#                             ]),}

In [5]:
def create_split_train_val_test(train_dir, test_dir, transforms, val_size=0.15):
    train_data = datasets.ImageFolder(train_dir,       
                    transform=transforms)
    test_data = datasets.ImageFolder(test_dir,
                    transform=transforms)
    train_subset, val_subset = torch.utils.data.random_split(train_data, [1-val_size, val_size], generator=torch.Generator().manual_seed(1))
    torch.save(train_subset, 'train_data.pt')
    torch.save(val_subset, 'val_data.pt')
    torch.save(test_data, 'test_data.pt')

def load_data(dir, batch_size=64):
    train_data = torch.load(os.path.join(dir, 'train_data.pt'))
    val_data = torch.load(os.path.join(dir, 'val_data.pt'))
    test_data = torch.load(os.path.join(dir, 'test_data.pt'))
    train_loader = torch.utils.data.DataLoader(train_data, shuffle=True, batch_size=batch_size, num_workers=3)
    val_loader = torch.utils.data.DataLoader(val_data, shuffle=True, batch_size=batch_size, num_workers=3)
    test_loader = torch.utils.data.DataLoader(test_data, shuffle=False, batch_size=batch_size, num_workers=3)
    return train_loader, val_loader, test_loader

In [6]:
create_split_train_val_test(train_dir, test_dir, cnn_transforms)

In [7]:
train_loader, val_loader, test_loader = load_data(mydir, 64)

C:\Users\rohil\AppData\Local\Temp\ipykernel_13828\3774757452.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  train_data = torch.load(os.path.join(dir, 'train_data.pt'))

In [8]:
dataloaders = {"train":train_loader, "val":val_loader}
data_sizes = {x: len(dataloaders[x].sampler) for x in ['train','val']}

In [9]:
class_names = train_loader.dataset.dataset.classes

## Model Train Function

In [10]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=10):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = np.inf

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            current_loss = 0.0
            current_corrects = 0

            for inputs, labels in tqdm.tqdm(dataloaders[phase], desc=phase, leave=False):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # We need to zero the gradients in the Cache.
                optimizer.zero_grad()

                # Time to carry out the forward training poss
                # We only need to log the loss stats if we are in training phase
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                if phase == 'train':
                    scheduler.step()

                # We want variables to hold the loss statistics
                current_loss += loss.item() * inputs.size(0)
                current_corrects += torch.sum(preds == labels.data)
            epoch_loss = current_loss / data_sizes[phase]
            epoch_acc = current_corrects.double() / data_sizes[phase]
            if phase == 'val':
                print('{} Loss: {:.4f} | {} Accuracy: {:.4f}'.format(
                    phase, epoch_loss, phase, epoch_acc))
            else:
                print('{} Loss: {:.4f} | {} Accuracy: {:.4f}'.format(
                    phase, epoch_loss, phase, epoch_acc))

            # EARLY STOPPING
            if phase == 'val' and epoch_loss < best_loss:
                print('Val loss Decreased from {:.4f} to {:.4f} \nSaving Weights... '.format(best_loss, epoch_loss))
                best_loss = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_since = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_since // 60, time_since % 60))
    print('Best val loss: {:.4f}'.format(best_loss))

    # Now we'll load in the best model weights and return it
    model.load_state_dict(best_model_wts)
    return model

## Dummy Model

In [11]:
class DummyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear = nn.Linear(3*224*224,2)
    
    def forward(self, input):
        output = self.flatten(input)
        output = self.linear(output)
        return output

In [12]:
def dummy():
    model = DummyModel()
    model = model.to(device)
    return model

dummy_model = dummy()

# specify loss function (categorical cross-entropy loss)
criterion = nn.CrossEntropyLoss() 

# Specify optimizer which performs Gradient Descent
optimizer = optim.Adam(dummy_model.parameters(), lr=1e-3)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1) # Learning Scheduler

In [13]:
best_dummy_model = train_model(dummy_model, criterion, optimizer, exp_lr_scheduler, num_epochs=10)

Epoch 1/10
----------


train:   0%|          | 0/70 [00:00<?, ?it/s]

train Loss: 13.4141 | train Accuracy: 0.8024


val:   0%|          | 0/13 [00:00<?, ?it/s]

val Loss: 9.4116 | val Accuracy: 0.8406
Val loss Decreased from inf to 9.4116 
Saving Weights... 

Epoch 2/10
----------


...

val Loss: 9.4116 | val Accuracy: 0.8406

Epoch 10/10
----------


train:   0%|          | 0/70 [00:00<?, ?it/s]

train Loss: 11.5953 | train Accuracy: 0.8174


val:   0%|          | 0/13 [00:00<?, ?it/s]

val Loss: 9.4116 | val Accuracy: 0.8406
Val loss Decreased from 9.4116 to 9.4116 
Saving Weights... 

Training complete in 9m 47s
Best val loss: 9.4116


In [14]:
y_pred_list = []
y_true_list = []
test_loss = 0.0
test_corrects = 0

with torch.no_grad():
    for x_batch, y_batch in tqdm.tqdm(test_loader, leave=False):
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        y_test_pred = best_dummy_model(x_batch)
        y_test_pred = torch.log_softmax(y_test_pred, dim=1)
        _, y_pred_tag = torch.max(y_test_pred, dim = 1)
        loss = criterion(y_test_pred, y_batch)
        test_loss += loss.item() * x_batch.size(0)
        test_corrects += torch.sum(y_pred_tag == y_batch.data)

test_loss = test_loss / len(test_loader.sampler)
test_acc = test_corrects.double() / len(test_loader.sampler)

  0%|          | 0/10 [00:00<?, ?it/s]

In [15]:
print(test_loss)
print(test_acc)

12.460241024310772
tensor(0.8173, device='cuda:0', dtype=torch.float64)


In [16]:
torch.save(best_dummy_model.state_dict(), 'best_dummy_model.pth')

## Densenet Model

In [17]:
def Densenet_Model(class_names, pretrained=True):
    if pretrained:
        model = models.densenet121(weights='DEFAULT')
    else:
        model = models.densenet121(weights=None)

    for params in model.parameters():
        params.requires_grad=False
    num_ftrs = model.classifier.in_features
    model.classifier = nn.Linear(num_ftrs, len(class_names))
    model = model.to(device)
    return model

dense_model = Densenet_Model(class_names, pretrained=True)

# specify loss function (categorical cross-entropy loss)
criterion = nn.CrossEntropyLoss() 

# Specify optimizer which performs Gradient Descent
optimizer = optim.Adam(dense_model.parameters(), lr=1e-3)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1) # Learning Scheduler

In [18]:
pytorch_total_params = sum(p.numel() for p in dense_model.parameters() if p.requires_grad)
print("Number of trainable parameters: \n{}".format(pytorch_total_params))

Number of trainable parameters: 
2050


In [19]:
best_densenet_model = train_model(dense_model, criterion, optimizer, exp_lr_scheduler, num_epochs=10)

Epoch 1/10
----------


train:   0%|          | 0/70 [00:00<?, ?it/s]

train Loss: 0.4868 | train Accuracy: 0.7563


val:   0%|          | 0/13 [00:00<?, ?it/s]

val Loss: 0.4712 | val Accuracy: 0.7487
Val loss Decreased from inf to 0.4712 
Saving Weights... 

Epoch 2/10
----------


...

val Loss: 0.4688 | val Accuracy: 0.7589

Epoch 10/10
----------


train:   0%|          | 0/70 [00:00<?, ?it/s]

train Loss: 0.4738 | train Accuracy: 0.7633


val:   0%|          | 0/13 [00:00<?, ?it/s]

val Loss: 0.4680 | val Accuracy: 0.7577

Training complete in 11m 2s
Best val loss: 0.4658


In [20]:
y_pred_list = []
y_true_list = []
test_loss = 0.0
test_corrects = 0

with torch.no_grad():
    for x_batch, y_batch in tqdm.tqdm(test_loader, leave=False):
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        y_test_pred = best_densenet_model(x_batch)
        y_test_pred = torch.log_softmax(y_test_pred, dim=1)
        _, y_pred_tag = torch.max(y_test_pred, dim = 1)
        loss = criterion(y_test_pred, y_batch)
        test_loss += loss.item() * x_batch.size(0)
        test_corrects += torch.sum(y_pred_tag == y_batch.data)

test_loss = test_loss / len(test_loader.sampler)
test_acc = test_corrects.double() / len(test_loader.sampler)

  0%|          | 0/10 [00:00<?, ?it/s]

In [21]:
print(test_loss)
print(test_acc)

0.5860501329104105
tensor(0.6346, device='cuda:0', dtype=torch.float64)


In [22]:
torch.save(best_densenet_model.state_dict(), 'best_densenet_model.pth')

## Resnet Model

In [23]:
def Resnet_Model(class_names, pretrained=True):
    if pretrained:
        model = models.resnet50(weights='DEFAULT')
    else:
        model = models.resnet50(weights=None)
    for params in model.parameters():
        params.requires_grad=False
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, len(class_names))
    model = model.to(device)
    return model

res_model = Resnet_Model(class_names, pretrained=True)

# specify loss function (categorical cross-entropy loss)
criterion = nn.CrossEntropyLoss() 

# Specify optimizer which performs Gradient Descent
optimizer = optim.Adam(res_model.parameters(), lr=1e-3)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1) # Learning Scheduler

In [24]:
pytorch_total_params = sum(p.numel() for p in res_model.parameters() if p.requires_grad)
print("Number of trainable parameters: \n{}".format(pytorch_total_params))

Number of trainable parameters: 
4098


In [25]:
best_resnet_model = train_model(res_model, criterion, optimizer, exp_lr_scheduler, num_epochs=10)

Epoch 1/10
----------


train:   0%|          | 0/70 [00:00<?, ?it/s]

train Loss: 0.4945 | train Accuracy: 0.7385


val:   0%|          | 0/13 [00:00<?, ?it/s]

val Loss: 0.5467 | val Accuracy: 0.7411
Val loss Decreased from inf to 0.5467 
Saving Weights... 

Epoch 2/10
----------


...

val Loss: 0.4819 | val Accuracy: 0.7411
Val loss Decreased from 0.4827 to 0.4819 
Saving Weights... 

Epoch 10/10
----------


train:   0%|          | 0/70 [00:00<?, ?it/s]

train Loss: 0.4841 | train Accuracy: 0.7424


val:   0%|          | 0/13 [00:00<?, ?it/s]

val Loss: 0.4851 | val Accuracy: 0.7411

Training complete in 11m 22s
Best val loss: 0.4819


In [26]:
y_pred_list = []
y_true_list = []
test_loss = 0.0
test_corrects = 0

with torch.no_grad():
    for x_batch, y_batch in tqdm.tqdm(test_loader, leave=False):
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        y_test_pred = best_resnet_model(x_batch)
        y_test_pred = torch.log_softmax(y_test_pred, dim=1)
        _, y_pred_tag = torch.max(y_test_pred, dim = 1)
        loss = criterion(y_test_pred, y_batch)
        test_loss += loss.item() * x_batch.size(0)
        test_corrects += torch.sum(y_pred_tag == y_batch.data)

test_loss = test_loss / len(test_loader.sampler)
test_acc = test_corrects.double() / len(test_loader.sampler)

  0%|          | 0/10 [00:00<?, ?it/s]

In [27]:
print(test_loss)
print(test_acc)

0.635118889885071
tensor(0.6250, device='cuda:0', dtype=torch.float64)


In [28]:
torch.save(best_resnet_model.state_dict(), 'best_resnet_model.pth')

## Transformer

In [29]:
def Transformer_Model(class_names, pretrained=True):
    if pretrained:
        model = models.vit_b_16(weights='DEFAULT')
    else:
        model = models.resnet50(weights=None)

    weights = models.ViT_B_16_Weights.DEFAULT
    transforms = weights.transforms()
    #Freeze transformer
    for params in model.parameters():
        params.requires_grad=False
    
    num_ftrs = model.hidden_dim
    model.heads = nn.Linear(num_ftrs, len(class_names))
    model = model.to(device)
    return model, transforms

transformer_model, transformer_transforms = Transformer_Model(class_names, pretrained=True)

# specify loss function (categorical cross-entropy loss)
criterion = nn.CrossEntropyLoss() 

# Specify optimizer which performs Gradient Descent
optimizer = optim.Adam(transformer_model.parameters(), lr=1e-3)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1) # Learning Scheduler

In [30]:
transformer_transforms

ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BILINEAR
)

In [31]:
pytorch_total_params = sum(p.numel() for p in transformer_model.parameters() if p.requires_grad)
print("Number of trainable parameters: \n{}".format(pytorch_total_params))

Number of trainable parameters: 
1538


In [32]:
create_split_train_val_test(train_dir, test_dir, transformer_transforms)

In [33]:
train_loader, val_loader, test_loader = load_data(mydir, 64)

C:\Users\rohil\AppData\Local\Temp\ipykernel_13828\3774757452.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  train_data = torch.load(os.path.join(dir, 'train_data.pt'))

In [34]:
dataloaders = {"train":train_loader, "val":val_loader}
data_sizes = {x: len(dataloaders[x].sampler) for x in ['train','val']}

In [35]:
best_transformer_model = train_model(transformer_model, criterion, optimizer, exp_lr_scheduler, num_epochs=10)

Epoch 1/10
----------


train:   0%|          | 0/70 [00:00<?, ?it/s]

c:\Users\rohil\anaconda3\envs\cs7643-a3\Lib\site-packages\torch\nn\functional.py:5560: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = scaled_dot_product_attention(q, k, v, attn_mask, dropout_p, is_causal)


train Loss: 0.3177 | train Accuracy: 0.8934


val:   0%|          | 0/13 [00:00<?, ?it/s]

...

val Loss: 0.2980 | val Accuracy: 0.9094
Val loss Decreased from 0.2980 to 0.2980 
Saving Weights... 

Epoch 10/10
----------


train:   0%|          | 0/70 [00:00<?, ?it/s]

train Loss: 0.3019 | train Accuracy: 0.9080


val:   0%|          | 0/13 [00:00<?, ?it/s]

val Loss: 0.2980 | val Accuracy: 0.9094

Training complete in 23m 60s
Best val loss: 0.2980


In [36]:
y_pred_list = []
y_true_list = []
test_loss = 0.0
test_corrects = 0

with torch.no_grad():
    for x_batch, y_batch in tqdm.tqdm(test_loader, leave=False):
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        y_test_pred = best_transformer_model(x_batch)
        y_test_pred = torch.log_softmax(y_test_pred, dim=1)
        _, y_pred_tag = torch.max(y_test_pred, dim = 1)
        loss = criterion(y_test_pred, y_batch)
        test_loss += loss.item() * x_batch.size(0)
        test_corrects += torch.sum(y_pred_tag == y_batch.data)

test_loss = test_loss / len(test_loader.sampler)
test_acc = test_corrects.double() / len(test_loader.sampler)

  0%|          | 0/10 [00:00<?, ?it/s]

In [37]:
print(test_loss)
print(test_acc)

0.4429000127009856
tensor(0.7804, device='cuda:0', dtype=torch.float64)


In [38]:
torch.save(best_transformer_model.state_dict(), 'best_transformer_model.pth')

## Hybrid Resnet + Transformer

In [39]:
class hybrid_model(nn.Module):
    def __init__(self, vit_model, resnet_model, dropout=0.3, device=device):
        super().__init__()
        self.vit_model = vit_model.to(device)
        self.vit_linear = nn.Linear(768, 2048, device=device)

        self.resnet_model = resnet_model.to(device)

        self.dropout = nn.Dropout(dropout)
        self.linear1 = nn.Linear(4096, 500, device=device)
        self.linear2 = nn.Linear(500, 2, device=device)

    def forward(self, inputs):
        vit_result = self.vit_model(inputs)['getitem_5']
        vit_result = self.vit_linear(vit_result)

        res_result = self.resnet_model(inputs)
        res_result = torch.reshape(res_result, (res_result.shape[0], res_result.shape[1]))

        output = torch.cat((vit_result, res_result), 1)
        output = self.dropout(output)
        output = self.linear1(output)
        output = self.dropout(output)
        output = self.linear2(output)
        return output


In [40]:
def hybrid(class_names):
    resnet_model = models.resnet50(weights='DEFAULT')
    vit_model = models.vit_b_16(weights='DEFAULT')
    
    # Remove the last layer of Resnet
    resnet_layers = list(resnet_model.children())
    resnet_model = nn.Sequential(*resnet_layers[:-1])

    for params in resnet_model.parameters():
        params.requires_grad=False


    for params in vit_model.parameters():
        params.requires_grad=False

    vit_model_top = create_feature_extractor(vit_model, return_nodes=['getitem_5'])
    model = hybrid_model(vit_model_top, resnet_model, 0.3, device)

    model = model.to(device)
    return model

hmodel = hybrid(class_names)

# specify loss function (categorical cross-entropy loss)
criterion = nn.CrossEntropyLoss() 

# Specify optimizer which performs Gradient Descent
optimizer = optim.Adam(hmodel.parameters(), lr=1e-3)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1) # Learning Scheduler

In [41]:
full_train_df = {}
labels = ['PNEUMONIA', 'NORMAL']
i = 0
for label in labels:
    path = os.path.join(train_dir, label)
    for img in os.listdir(path):
        full_train_df[i] = [os.path.join(path, img), label]
        i += 1

full_train_df = pd.DataFrame.from_dict(data=full_train_df, orient='index', columns=['file_path','labels'])
train_df = full_train_df.sample(frac=0.85, random_state=42).sample(frac=1.0, random_state=42)
val_df = full_train_df.drop(train_df.index).sample(frac=1.0, random_state=42)


test_df = {}
i = 0
for label in labels:
    path = os.path.join(test_dir, label)
    for img in os.listdir(path):
        test_df[i] = [os.path.join(path, img), label]
        i += 1

test_df = pd.DataFrame.from_dict(data=test_df, orient='index', columns=['file_path','labels'])

In [42]:
create_split_train_val_test(train_dir, test_dir, transformer_transforms)

In [43]:
train_loader, val_loader, test_loader = load_data(mydir, 64)

C:\Users\rohil\AppData\Local\Temp\ipykernel_13828\3774757452.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  train_data = torch.load(os.path.join(dir, 'train_data.pt'))

In [44]:
dataloaders = {"train":train_loader, "val":val_loader}
data_sizes = {x: len(dataloaders[x].sampler) for x in ['train','val']}

In [45]:
pytorch_total_params = sum(p.numel() for p in hmodel.parameters() if p.requires_grad)
print("Number of trainable parameters: \n{}".format(pytorch_total_params))

Number of trainable parameters: 
3624414


In [46]:
def train_hybrid_model(model, criterion, optimizer, scheduler, num_epochs=10):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = np.inf

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            current_loss = 0.0
            current_corrects = 0

            for inputs, labels in tqdm.tqdm(dataloaders[phase], desc=phase, leave=False):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # We need to zero the gradients in the Cache.
                optimizer.zero_grad()

                # Time to carry out the forward training poss
                # We only need to log the loss stats if we are in training phase
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                if phase == 'train':
                    scheduler.step()

                # We want variables to hold the loss statistics
                current_loss += loss.item() * inputs.size(0)
                current_corrects += torch.sum(preds == labels.data)
            epoch_loss = current_loss / data_sizes[phase]
            epoch_acc = current_corrects.double() / data_sizes[phase]
            if phase == 'val':
                print('{} Loss: {:.4f} | {} Accuracy: {:.4f}'.format(
                    phase, epoch_loss, phase, epoch_acc))
            else:
                print('{} Loss: {:.4f} | {} Accuracy: {:.4f}'.format(
                    phase, epoch_loss, phase, epoch_acc))

            # EARLY STOPPING
            if phase == 'val' and epoch_loss < best_loss:
                print('Val loss Decreased from {:.4f} to {:.4f} \nSaving Weights... '.format(best_loss, epoch_loss))
                best_loss = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_since = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_since // 60, time_since % 60))
    print('Best val loss: {:.4f}'.format(best_loss))

    # Now we'll load in the best model weights and return it
    model.load_state_dict(best_model_wts)
    return model

In [47]:
best_hybrid_model = train_hybrid_model(hmodel, criterion, optimizer, exp_lr_scheduler)

Epoch 1/10
----------


train:   0%|          | 0/70 [00:00<?, ?it/s]

train Loss: 0.3748 | train Accuracy: 0.9049


val:   0%|          | 0/13 [00:00<?, ?it/s]

val Loss: 0.2769 | val Accuracy: 0.9260
Val loss Decreased from inf to 0.2769 
Saving Weights... 

Epoch 2/10
----------


...

val Loss: 0.2782 | val Accuracy: 0.9286

Epoch 10/10
----------


train:   0%|          | 0/70 [00:00<?, ?it/s]

train Loss: 0.2713 | train Accuracy: 0.9254


val:   0%|          | 0/13 [00:00<?, ?it/s]

val Loss: 0.2790 | val Accuracy: 0.9273

Training complete in 26m 21s
Best val loss: 0.2769


In [48]:
y_pred_list = []
y_true_list = []
test_loss = 0.0
test_corrects = 0

with torch.no_grad():
    for x_batch, y_batch in tqdm.tqdm(test_loader, leave=False):
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        y_test_pred = best_hybrid_model(x_batch)
        y_test_pred = torch.log_softmax(y_test_pred, dim=1)
        _, y_pred_tag = torch.max(y_test_pred, dim = 1)
        loss = criterion(y_test_pred, y_batch)
        test_loss += loss.item() * x_batch.size(0)
        test_corrects += torch.sum(y_pred_tag == y_batch.data)

test_loss = test_loss / len(test_loader.sampler)
test_acc = test_corrects.double() / len(test_loader.sampler)

  0%|          | 0/10 [00:00<?, ?it/s]

In [49]:
print(test_loss)
print(test_acc)

0.803626088377757
tensor(0.8333, device='cuda:0', dtype=torch.float64)


In [50]:
torch.save(best_hybrid_model.state_dict(), 'best_hybrid_model.pth')